In [1]:
import sys
import os.path
import json
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
from itertools import product
import gc
import re
from catboost import CatBoostRegressor, Pool
from utils import load_feature_set, clipped_rmse, HoldOut
import pickle
import joblib

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive') 
  if not os.path.isfile('SETTINGS.json'):
       # hard coded data directory in drive is used if SETTINGS.json not present 
       config={}
       config['DATA_DIR'] = '/content/gdrive/My Drive/kaggle-c1'
       with open('SETTINGS.json', 'w') as outfile:
         json.dump(config, outfile)

with open('SETTINGS.json') as config_file:
    config = json.load(config_file)

DATA_DIR = config['DATA_DIR']

print('Using DATA_DIR ', DATA_DIR)

DATA_FOLDER = DATA_DIR

Using DATA_DIR  /home/vvi/repos/c1-final-test/datadir


In [2]:
# read feature selection and hyperparameter search results from disk

with open(os.path.join(DATA_FOLDER,'search_path_sfs.pickle'), 'rb') as fp:    
            search_path_sfs = pickle.load(fp)
        
with open(os.path.join(DATA_FOLDER,'best_params_catboost.json')) as fp:        
    best_params_catboost = json.load(fp)

        
with open(os.path.join(DATA_FOLDER,'best_params_rf.json')) as fp:        
    best_params_rf = json.load(fp)
        

In [3]:
X_trainval, y_trainval = load_feature_set('allfeat', data_folder=DATA_FOLDER, datasets='trainval')

reading from file /home/vvi/repos/c1-final-test/datadir/feature_set_allfeat.csv


## CatBoost model training

In [4]:
%%time
for id in ['basic','basicv2','allfeat','text','within']:
  for numsel in [9,16,25]:  
    print('generating model_catboost_{}_sel{}'.format(id,numsel))
    print('keeping cols ', list(search_path_sfs[id][numsel]['feature_names']))
    to_drop_cols= [col for col in X_trainval.columns.values if not col in list(search_path_sfs[id][numsel]['feature_names'])]
    reg=CatBoostRegressor(**best_params_catboost['{}_sel{}'.format(id,numsel)])
    reg.fit(X_trainval.drop(to_drop_cols,axis=1).to_numpy(), y_trainval)
    joblib_filename = os.path.join(DATA_FOLDER,'model_catboost_{}_sel{}.joblib'.format(id,numsel))
    joblib.dump(reg, joblib_filename+'.bz2', compress=('bz2', 3))
    
    #pred_test = np.clip(reg.predict(X_test.drop(to_drop_cols,axis=1).to_numpy()), 0, 20)
    #output_name = 'submission-rf-bestparams-features_{}_combination_{}.csv'.format(id,'sel{}'.format(numsel))
    #write_predictions_by_array(pred_test[submissionidx2testidx], output_name)  


generating model_catboost_basic_sel9
keeping cols  ['target_item_lag_4', 'target_lag_2', 'item_category_id', 'target_item_lag_2', 'target_shop_lag_12', 'target_category_lag_4', 'target_item_lag_12', 'item_name_category_tfidf_bigram_256', 'time_of_year']
0:	learn: 1.1862077	total: 192ms	remaining: 3m 12s
1:	learn: 1.1862078	total: 226ms	remaining: 1m 52s
2:	learn: 1.1758468	total: 407ms	remaining: 2m 15s
3:	learn: 1.1758467	total: 442ms	remaining: 1m 50s
4:	learn: 1.1661137	total: 620ms	remaining: 2m 3s
5:	learn: 1.1661135	total: 653ms	remaining: 1m 48s
6:	learn: 1.1567037	total: 832ms	remaining: 1m 57s
7:	learn: 1.1567036	total: 866ms	remaining: 1m 47s
8:	learn: 1.1476054	total: 1.04s	remaining: 1m 54s
9:	learn: 1.1476054	total: 1.07s	remaining: 1m 46s
10:	learn: 1.1393399	total: 1.25s	remaining: 1m 52s
11:	learn: 1.1393398	total: 1.28s	remaining: 1m 45s
12:	learn: 1.1313950	total: 1.46s	remaining: 1m 51s
13:	learn: 1.1313949	total: 1.5s	remaining: 1m 45s
14:	learn: 1.1238025	total: 1.

### Training time of catboost models

Real time needed to train all the catboost models was 25 minutes on a NVIDIA GTX 1050 GPU (i5-3570 CPU).  
The time taken to train the three models that were actually used in the final submission ensemble was 1 min 59s (i.e. 'model_catboost_allfeat_sel16', 'model_catboost_basic_sel25', 'model_catboost_basicv2_sel9', see Notebook 8).

## Random forest model training

In [5]:
%%time
from sklearn.ensemble import RandomForestRegressor

for id in ['basic','basicv2','allfeat','text','within']:
  for numsel in [9,16,25]:  
    print('generating model_rf_{}_sel{}'.format(id,numsel))
    print('keeping cols ', list(search_path_sfs[id][numsel]['feature_names']))
    to_drop_cols= [col for col in X_trainval.columns.values if not col in list(search_path_sfs[id][numsel]['feature_names'])]
    reg=RandomForestRegressor(verbose=2, n_jobs=-1, **best_params_rf['{}_sel{}'.format(id,numsel)])
    reg.fit(X_trainval.drop(to_drop_cols,axis=1).to_numpy(), y_trainval)
    joblib_filename = os.path.join(DATA_FOLDER,'model_rf_{}_sel{}.joblib'.format(id,numsel))
    joblib.dump(reg, joblib_filename+'.bz2', compress=('bz2', 3))
   

generating model_rf_basic_sel9
keeping cols  ['target_item_lag_4', 'target_lag_2', 'item_category_id', 'target_item_lag_2', 'target_shop_lag_12', 'target_category_lag_4', 'target_item_lag_12', 'item_name_category_tfidf_bigram_256', 'time_of_year']
building tree 1 of 100building tree 2 of 100

building tree 3 of 100building tree 4 of 100



[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  3.0min


building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78

...

building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 10.6min


building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78

[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 29.7min finished


CPU times: user 10h 14min 22s, sys: 1min 3s, total: 10h 15min 26s
Wall time: 2h 42min 8s


Public LB performance:

|Feature set / subset | sel9 | sel16 | sel25 |
|---|---|---|---|
|allfeat|  1.03695| 0.98305|0.993841 |
|basic| 1.012312 | 1.011059| 1.003010 |
|basicv2| 0.96716 | 0.99535| 0.99384 |

### Training time of random forest models

Real time needed to train all the random forest models was 2h 28 min on a i5-3570 CPU with 4 cores running at 3.4GHz (on an earlier run). 
The time taken to train the three models that were actually used in the final submission ensemble (i.e. 'model_rf_allfeat_sel16','model_rf_basic_sel25','model_rf_basicv2_sel25', see Notebook 8) was 24.7 minutes.